In [14]:
import pandas as pd

# sales.csv
sales = pd.DataFrame({
    'OrderID': [101, 102, 103, 104, 105],
    'CustID': ['C001', 'C002', 'C001', 'C003', 'C002'],
    'ProductID': ['P001', 'P002', 'P003', 'P002', 'P001'],
    'Quantity': [5, 3, 2, 1, 4],
    'OrderDate': ['2025-11-01', '2025-11-01', '2025-11-02', '2025-11-02', '2025-11-03']
})

# products.csv
products = pd.DataFrame({
    'ProductID': ['P001', 'P002', 'P003', 'P004'],
    'ProductName': ['Keyboard', 'Mouse', 'Monitor', 'Office Chair'],
    'Category': ['Electronics', 'Electronics', 'Electronics', 'Furniture'],
    'Price': [50, 20, 150, 200]
})

# customers.csv
customers = pd.DataFrame({
    'CustomerID': ['C001', 'C002', 'C003', 'C004'],
    'CustomerName': ['John Doe', 'Jane Smith', 'Emily Davis', 'Michael Brown'],
    'City': ['New York', 'Chicago', 'Los Angeles', 'Miami']
})


Questions on groupby

1. Using the sales.csv file, find the total quantity of products ordered by each CustomerID.

2. Group the sales data by OrderDate and calculate the average quantity of products ordered each day.

3. For each product in sales.csv, calculate the total quantity sold and the number of distinct customers who bought it.

In [15]:
sales.head()

,OrderID,CustID,ProductID,Quantity,OrderDate
0,101,C001,P001,5,2025-11-01
1,102,C002,P002,3,2025-11-01
2,103,C001,P003,2,2025-11-02
3,104,C003,P002,1,2025-11-02
4,105,C002,P001,4,2025-11-03


In [16]:
products.head()

,ProductID,ProductName,Category,Price
0,P001,Keyboard,Electronics,50
1,P002,Mouse,Electronics,20
2,P003,Monitor,Electronics,150
3,P004,Office Chair,Furniture,200


In [17]:
customers.head()

,CustomerID,CustomerName,City
0,C001,John Doe,New York
1,C002,Jane Smith,Chicago
2,C003,Emily Davis,Los Angeles
3,C004,Michael Brown,Miami


In [54]:
# 1. Using the sales.csv file, find the total quantity of products ordered by each CustomerID.
res = sales.groupby("CustID")["Quantity"].sum().reset_index()
df = pd.merge(res , customers , left_on="CustID" , right_on="CustomerID" , how="inner")
df.loc[:,["CustID" , "CustomerName" , "Quantity"]]

,CustID,CustomerName,Quantity
0,C001,John Doe,7
1,C002,Jane Smith,7
2,C003,Emily Davis,1


In [55]:
# 2 Group the sales data by OrderDate and calculate the average quantity of products ordered each day.
res = sales.groupby("OrderDate")["Quantity"].mean().reset_index()
res

,OrderDate,Quantity
0,2025-11-01,4.0
1,2025-11-02,1.5
2,2025-11-03,4.0


In [56]:
# 3 For each product in sales.csv, calculate the total quantity sold and the number of distinct customers who bought it.
res1 = sales.groupby("ProductID")["Quantity"].sum()
res2 = sales.groupby("ProductID")["CustID"].count()
res = pd.merge(res1 , res2 , on = "ProductID" , how="inner")
res

,Quantity,CustID
ProductID,,
P001,9,2
P002,4,2
P003,2,1


Questions on merge
1. Merge sales.csv and products.csv on ProductID to get a combined table with product details included for each sale.

2. Merge sales.csv with customers.csv to associate each order with customer information.

3. Perform an outer merge between customers.csv and sales.csv to list all customers, including those who have not made any orders.

In [57]:
# 1. Merge sales.csv and products.csv on ProductID to get a combined table with product details included for each sale.
pd.merge(sales , products , on = "ProductID" , how="inner")

,OrderID,CustID,ProductID,Quantity,OrderDate,ProductName,Category,Price
0,101,C001,P001,5,2025-11-01,Keyboard,Electronics,50
1,102,C002,P002,3,2025-11-01,Mouse,Electronics,20
2,103,C001,P003,2,2025-11-02,Monitor,Electronics,150
3,104,C003,P002,1,2025-11-02,Mouse,Electronics,20
4,105,C002,P001,4,2025-11-03,Keyboard,Electronics,50


In [59]:
# 2. Merge sales.csv with customers.csv to associate each order with customer information.
pd.merge(sales , customers , left_on = "CustID" , right_on = "CustomerID" , how="inner")

,OrderID,CustID,ProductID,Quantity,OrderDate,CustomerID,CustomerName,City
0,101,C001,P001,5,2025-11-01,C001,John Doe,New York
1,102,C002,P002,3,2025-11-01,C002,Jane Smith,Chicago
2,103,C001,P003,2,2025-11-02,C001,John Doe,New York
3,104,C003,P002,1,2025-11-02,C003,Emily Davis,Los Angeles
4,105,C002,P001,4,2025-11-03,C002,Jane Smith,Chicago


In [61]:
# 3. Perform an outer merge between customers.csv and sales.csv to list all customers, including those who have not made any orders.
pd.merge(customers , sales , left_on = "CustomerID" , right_on = "CustID" , how="outer")


,CustomerID,CustomerName,City,OrderID,CustID,ProductID,Quantity,OrderDate
0,C001,John Doe,New York,101.0,C001,P001,5.0,2025-11-01
1,C001,John Doe,New York,103.0,C001,P003,2.0,2025-11-02
2,C002,Jane Smith,Chicago,102.0,C002,P002,3.0,2025-11-01
3,C002,Jane Smith,Chicago,105.0,C002,P001,4.0,2025-11-03
4,C003,Emily Davis,Los Angeles,104.0,C003,P002,1.0,2025-11-02
5,C004,Michael Brown,Miami,NaN,NaN,NaN,NaN,NaN


Questions on join
1. Using the products.csv as the left DataFrame, perform a join with sales.csv on ProductID to find sales details for each product.

2. Join customers.csv with sales.csv on CustomerID but keep only customers who have made orders (inner join).

3. Perform a left join of sales.csv with products.csv to show all sales and product details, including sales with missing product info if any.

In [92]:
# 1. Using the products.csv as the left DataFrame, perform a join with sales.csv on ProductID to find sales details for each product.
dfp = products.set_index("ProductID")
dfs = sales.set_index("ProductID")

dfp.join(dfs , how="left").reset_index()

,ProductID,ProductName,Category,Price,OrderID,CustID,Quantity,OrderDate
0,P001,Keyboard,Electronics,50,101.0,C001,5.0,2025-11-01
1,P001,Keyboard,Electronics,50,105.0,C002,4.0,2025-11-03
2,P002,Mouse,Electronics,20,102.0,C002,3.0,2025-11-01
3,P002,Mouse,Electronics,20,104.0,C003,1.0,2025-11-02
4,P003,Monitor,Electronics,150,103.0,C001,2.0,2025-11-02
5,P004,Office Chair,Furniture,200,NaN,NaN,NaN,NaN


In [94]:
# 2. Join customers.csv with sales.csv on CustomerID but keep only customers who have made orders (inner join).
dfc = customers.set_index("CustomerID")
dfs = sales.set_index("CustID")

dfc.join(dfs , how="inner").reset_index()

,CustomerID,CustomerName,City,OrderID,ProductID,Quantity,OrderDate
0,C001,John Doe,New York,101,P001,5,2025-11-01
1,C001,John Doe,New York,103,P003,2,2025-11-02
2,C002,Jane Smith,Chicago,102,P002,3,2025-11-01
3,C002,Jane Smith,Chicago,105,P001,4,2025-11-03
4,C003,Emily Davis,Los Angeles,104,P002,1,2025-11-02


In [97]:
# 3. Perform a left join of sales.csv with products.csv to show all sales and product details, including sales with missing product info if any.
dfs = sales.set_index("ProductID")
dfp = products.set_index("ProductID")

dfs.join(dfp , how="left").reset_index()

,ProductID,OrderID,CustID,Quantity,OrderDate,ProductName,Category,Price
0,P001,101,C001,5,2025-11-01,Keyboard,Electronics,50
1,P002,102,C002,3,2025-11-01,Mouse,Electronics,20
2,P003,103,C001,2,2025-11-02,Monitor,Electronics,150
3,P002,104,C003,1,2025-11-02,Mouse,Electronics,20
4,P001,105,C002,4,2025-11-03,Keyboard,Electronics,50


In [81]:
def print_butterfly(n):
  for i in range(n*2):
    if i<n:
      print("*"*(i+1)+" "*(2*(n-i-1))+"*"*(i+1))
    if i>=n:
      print("*"*(2*n-(i+1))+" "*(2*(i-n+1))+"*"*(2*n-(i+1)))

print_butterfly(6)

*          *
**        **
***      ***
****    ****
*****  *****
************
*****  *****
****    ****
***      ***
**        **
*          *
            
